## Library

In [ ]:
# !pip install tensorflow_hub
# !pip install tensorflow
# !pip install pandas
# !pip install numpy
# !pip install glob
# !pip install os
# !pip install tqdm
# !pip install matplotlib

In [1]:
from tqdm import tqdm

# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time
import json 

# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

2022-11-14 20:10:08.686193: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-14 20:10:08.852849: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-14 20:10:08.924602: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-14 20:10:09.579708: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

2.10.0
The following GPU devices are available: /device:GPU:0


2022-11-14 20:10:10.518964: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-14 20:10:10.541120: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 20:10:10.562583: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 20:10:10.562680: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA

## Implement

In [2]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

detector = hub.load(module_handle).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
2022-11-14 20:10:19.944670: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 20:10:19.944805: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 20:10:19.944855: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 20:10:19.945583: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 20:10:19.945652: I t

In [3]:
'''
Inputs:
The input tensor is a tf.float32 tensor with shape [1, height, width, 3] with values in [0.0, 1.0].

Outputs:
The output dictionary contains:

detection_boxes: a tf.float32 tensor of shape [N, 4] containing bounding box coordinates in the following order: [ymin, xmin, ymax, xmax].
detection_class_entities: a tf.string tensor of shape [N] containing detection class names as Freebase MIDs.
detection_class_names: a tf.string tensor of shape [N] containing human-readable detection class names.
detection_class_labels: a tf.int64 tensor of shape [N] with class indices.
detection_scores: a tf.float32 tensor of shape [N] containing detection scores.
'''

def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

def run_detector(detector, path):
  class_names = []
  prob_ratio = []

  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  result = detector(converted_img)

  result = {key:value.numpy().tolist() for key,value in result.items()}

  # class_name = [i.decode("utf-8") for i in result['detection_class_entities']]

  for prob, boxes, class_name in zip(result["detection_scores"], result['detection_boxes'], result['detection_class_entities']):
    class_name = class_name.decode("utf-8")
    class_names.append(class_name)

    area_ratio = (boxes[2]-boxes[0])*(boxes[3]-boxes[1])

    prob_ratio.append({class_name: [prob, area_ratio]})

  return class_names, prob_ratio, result['detection_boxes']
  
def load_json_file(json_path: str):
  with open(json_path, 'r') as f:
    js = json.loads(f.read())

  return {int(k):v for k,v in js.items()}

def get_ouput(video_id, frame_id, list_shot_id, counter, prob_ratio, bbox):
  res = {}
  res["Video_id"] = video_id
  res["Frame_id"] = frame_id
  res['List_shot_id'] = list_shot_id
  res['Collection'] = counter
  res['Prob_Ratio'] = prob_ratio
  res['Bbox'] = bbox
  return res

In [ ]:
import glob
import os
import collections
import pandas as pd

csv_content = []

################# Config Folder #################
output_csv_path = '/media/nhattuong/Data/OD_batch3_4.csv'
json_path = '/media/nhattuong/Data/AIC_2022/dict_support_model/dict_support_model_batch_3.json'
#################################################

id2img_fps = load_json_file(json_path)
num_imgs = len(id2img_fps)

for id, value in tqdm(id2img_fps.items()):
  # if id <= 201822: #94571 #161201
  #   continue
  
  image_path = '/media/nhattuong/Data/Video-Text-Retrieval/' + value["image_path"]
  class_names, prob_ratio, bbox = run_detector(detector, image_path)
  
  counter = dict(collections.Counter(class_names))

  video_id = f"{image_path.split('/')[-2]}"
  frame_id = f"{image_path.split('/')[-1]}"

  list_shot_id = value["list_shot_id"]

  csv_content.append(get_ouput(video_id, frame_id, list_shot_id, counter, prob_ratio, bbox))
  # print("[INFO: {}/{}] - Result {}".format(id, num_imgs, counter))

df = pd.DataFrame(csv_content)
df.to_csv(output_csv_path, index=False)

## Re Format

In [1]:
import json
from glob import glob

list_json = glob('/home/hoangtv/Desktop/disk_1TB/Tuong/Merge_Database/ClassesC*/*/*.json')
res = []
for json_path in list_json:
  info = json.load(open(json_path,'r'))
  video_id = json_path.split('/')[-2]
  frame_id = int(json_path.split('/')[-1][:-5])
  res.append({
      'video_id':video_id,
      'frame_id':frame_id,
      'collections':info
  })
import pandas as pd
df = pd.DataFrame(res)
df.to_csv('OD.csv', index=False)